In [1]:
import os
import json
import openai
from dotenv import load_dotenv

load_dotenv("../keys.env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
# 공통으로 사용하는 호출 함수
def call_openai(prompt, model="gpt-4o", max_tokens=1024):
    completion = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=0
    )
    return completion.choices[0].message.content

In [3]:
def step1(keyword):
    """
    Step 1:
    기업 비전 관점에서 주어진 키워드에 대한 명확한 정의와 평가 기준(제목, 가중치)을 JSON 형식으로 도출합니다.
    """
    prompt = f"""
기업의 비전은 회사가 왜 존재하는지, 미래에는 어떤 회사가 되고 싶은지에 대한 방향성을 제시합니다.

Step 1:
1. 기업의 비전 관점에서 '{keyword}'라는 키워드가 의미하는 바를 명확하고 구체적으로 정의하세요. 이 정의는 향후 평가 기준 설정의 기초가 됩니다.
2. 1단계에서 도출한 정의를 바탕으로 해당 기업의 '{keyword}'를 평가하기 위한 필수 평가 기준들을 도출하세요. 각 평가 기준은 제목과 가중치(예: 25%, 30% 등)로 구성되어야 합니다.
3. 입력되는 데이터는 기업의 홈페이지와 뉴스이기 때문에 반드시 기업의 관점에서 {keyword}에 대한 정의와 평가 기준을 도출해야 합니다.

출력은 아래 JSON 포맷을 따르세요:
{{
  "definition": "<여기에 '{keyword}'에 대한 정의 작성>",
  "criteria_list": [
      {{"title": "<평가 기준 제목>", "weight": "<가중치>"}},
      ...
  ]
}}
"""
    response_text = call_openai(prompt, model="gpt-4o", max_tokens=1024)
    try:
        output = json.loads(response_text)
    except Exception as e:
        print("Step 1 JSON 파싱 오류:", e)
        output = {}
    return output

In [4]:
def step2(keyword, step1_output):
    """
    Step 2:
    Step 1의 결과(정의와 평가 기준 목록)를 바탕으로, 각 평가 기준별로
    - 입력 데이터(예: 기업 홈페이지, 뉴스 기사 등)에서 추출하거나 추론해야 할 구체적 값 또는 지표
    - 1~5점의 객관적 정량 평가 기준(각 점수에 대한 기준 및 예시)
    를 JSON 형식으로 도출합니다.
    """
    definition = step1_output.get("definition", "")
    criteria_list = step1_output.get("criteria_list", "")
    
    prompt = f"""
Step 2:
주어진 키워드 '{keyword}'와 Step 1의 결과:
정의: {definition}
평가 기준 목록: {criteria_list}

각 평가 기준(위 JSON의 criteria_list 항목)에 대해 다음 사항을 상세하게 정의하세요:
1. 해당 평가 기준을 평가하기 위해 입력 데이터(예: 기업 홈페이지, 뉴스 기사 등)에서 추출하거나 추론해야 할
   구체적인 값, 텍스트 지표 또는 과정을 설명하세요.
2. 해당 평가 기준에 대해 1점부터 5점까지 객관적이고 정량적인 점수를 산출할 수 있도록, 각 점수(5,4,3,2,1)의
   기준 및 예시를 상세하게 기술하세요.

출력은 아래 JSON 포맷을 따르세요:
{{
  "criteria_details": [
      {{
         "title": "<평가 기준 제목>",
         "extraction_process": "<입력 데이터에서 추출 또는 추론할 사항에 대한 상세 설명>",
         "scoring": {{
             "5": "<5점 기준에 대한 설명>",
             "4": "<4점 기준에 대한 설명>",
             "3": "<3점 기준에 대한 설명>",
             "2": "<2점 기준에 대한 설명>",
             "1": "<1점 기준에 대한 설명>"
         }}
      }},
      ...
  ]
}}
"""
    response_text = call_openai(prompt, model="gpt-4o", max_tokens=1024)
    try:
        output = json.loads(response_text)
    except Exception as e:
        print("Step 2 JSON 파싱 오류:", e)
        output = {}
    return output

In [5]:
def step3(keyword, step1_output, step2_output):
    """
    Step 3:
    앞서 도출된 Step 1(정의 및 평가 기준 목록)와 Step 2(각 평가 기준별 상세 내용)를 바탕으로,
    최종 평가 프롬프트 템플릿을 구성합니다.
    템플릿에는 제목, 정의, 평가 기준, 최종 점수 산정 방법, 평가 시 유의사항이 포함되어야 합니다.
    """
    definition = step1_output.get("definition", "")
    criteria_list = step1_output.get("criteria_list", "")
    criteria_details = step2_output.get("criteria_details", "")
    
    prompt = f"""
Step 3:
앞서 도출된 결과들을 바탕으로 '{keyword}'에 대한 최종 평가 프롬프트 템플릿을 구성하세요.

최종 프롬프트 템플릿에는 다음 항목들이 모두 포함되어야 합니다:
1. 제목: '[키워드] 항목 평가 가이드' (예: '고객 항목 평가 가이드')
2. 정의: Step 1에서 도출된 '{keyword}'의 명확한 정의 -> {definition}
3. 텍스트 기반 객관적 평가 기준: Step 1의 평가 기준 목록({criteria_list})과
   Step 2의 각 평가 기준별 상세 내용({criteria_details})을 반영하여, 각 평가 기준(제목, 가중치, 세부 추출/점수 산출 기준)을 상세히 기술
4. 최종 점수 산정 방법: 각 평가 기준별 점수를 1~5점으로 산출한 후, 가중치에 따라 합산하는 방식으로 최종 점수를 산출합니다.
   예: (기준1 점수×가중치1) + (기준2 점수×가중치2) + ... , 최종 점수는 소수점 첫째 자리까지 표기
5. 평가 시 유의사항: 모든 키워드에 공통적으로 적용될 평가 시 유의사항
   (예: 텍스트에 명시된 내용 위주로 평가, 추론 배제, 객관적 수치와 구체적 사례 중시 등)

최종 프롬프트 템플릿은 하나의 완성된 텍스트 블록으로 출력하세요.
"""
    final_prompt = call_openai(prompt, model="gpt-4o", max_tokens=1024)
    return final_prompt

In [6]:
def generate_evaluation_prompt(keyword):
    # Step 1 실행
    step1_result = step1(keyword)
    print("=== Step 1 결과 ===")
    print(step1_result)
    
    # Step 2 실행
    step2_result = step2(keyword, step1_result)
    print("=== Step 2 결과 ===")
    print(step2_result)
    
    # Step 3 실행: 최종 평가 프롬프트 템플릿 생성
    final_prompt = step3(keyword, step1_result, step2_result)
    print("=== 최종 평가 프롬프트 템플릿 ===")
    print(final_prompt)
    
    return {
        "step1_result": step1_result,
        "step2_result": step2_result,
        "final_prompt": final_prompt
    }

In [7]:
keyword = "고객"
generate_evaluation_prompt(keyword)

Step 1 JSON 파싱 오류: Expecting value: line 1 column 1 (char 0)
=== Step 1 결과 ===
{}
Step 2 JSON 파싱 오류: Expecting value: line 1 column 1 (char 0)
=== Step 2 결과 ===
{}
=== 최종 평가 프롬프트 템플릿 ===
제목: '고객 항목 평가 가이드'

정의: '고객'은 제품이나 서비스를 구매하거나 이용하는 개인 또는 조직을 의미하며, 기업의 성공에 중요한 역할을 하는 이해관계자입니다.

텍스트 기반 객관적 평가 기준:
1. 고객 만족도 (가중치: 0.4)
   - 세부 추출/점수 산출 기준:
     - 고객의 긍정적 피드백 빈도
     - 고객 불만 사항의 신속한 해결 여부
     - 고객 재구매율
     - 점수 산출: 각 항목에 대해 1~5점으로 평가 후 평균 점수 산출

2. 고객 충성도 (가중치: 0.3)
   - 세부 추출/점수 산출 기준:
     - 고객의 브랜드 추천 의향
     - 장기 고객 비율
     - 고객 이탈률
     - 점수 산출: 각 항목에 대해 1~5점으로 평가 후 평균 점수 산출

3. 고객 피드백의 질 (가중치: 0.3)
   - 세부 추출/점수 산출 기준:
     - 피드백의 구체성 및 유용성
     - 피드백 수집의 체계성
     - 피드백 반영의 신속성
     - 점수 산출: 각 항목에 대해 1~5점으로 평가 후 평균 점수 산출

최종 점수 산정 방법: 각 평가 기준별 점수를 1~5점으로 산출한 후, 가중치에 따라 합산하는 방식으로 최종 점수를 산출합니다.
예: (고객 만족도 점수×0.4) + (고객 충성도 점수×0.3) + (고객 피드백의 질 점수×0.3), 최종 점수는 소수점 첫째 자리까지 표기

평가 시 유의사항:
- 텍스트에 명시된 내용 위주로 평가하며, 추론은 배제합니다.
- 객관적 수치와 구체적 사례를 중시하여 평가합니다.
- 평가자는 주관적 판단을 최소화하고, 명확한 근거에

{'step1_result': {},
 'step2_result': {},
 'final_prompt': "제목: '고객 항목 평가 가이드'\n\n정의: '고객'은 제품이나 서비스를 구매하거나 이용하는 개인 또는 조직을 의미하며, 기업의 성공에 중요한 역할을 하는 이해관계자입니다.\n\n텍스트 기반 객관적 평가 기준:\n1. 고객 만족도 (가중치: 0.4)\n   - 세부 추출/점수 산출 기준:\n     - 고객의 긍정적 피드백 빈도\n     - 고객 불만 사항의 신속한 해결 여부\n     - 고객 재구매율\n     - 점수 산출: 각 항목에 대해 1~5점으로 평가 후 평균 점수 산출\n\n2. 고객 충성도 (가중치: 0.3)\n   - 세부 추출/점수 산출 기준:\n     - 고객의 브랜드 추천 의향\n     - 장기 고객 비율\n     - 고객 이탈률\n     - 점수 산출: 각 항목에 대해 1~5점으로 평가 후 평균 점수 산출\n\n3. 고객 피드백의 질 (가중치: 0.3)\n   - 세부 추출/점수 산출 기준:\n     - 피드백의 구체성 및 유용성\n     - 피드백 수집의 체계성\n     - 피드백 반영의 신속성\n     - 점수 산출: 각 항목에 대해 1~5점으로 평가 후 평균 점수 산출\n\n최종 점수 산정 방법: 각 평가 기준별 점수를 1~5점으로 산출한 후, 가중치에 따라 합산하는 방식으로 최종 점수를 산출합니다.\n예: (고객 만족도 점수×0.4) + (고객 충성도 점수×0.3) + (고객 피드백의 질 점수×0.3), 최종 점수는 소수점 첫째 자리까지 표기\n\n평가 시 유의사항:\n- 텍스트에 명시된 내용 위주로 평가하며, 추론은 배제합니다.\n- 객관적 수치와 구체적 사례를 중시하여 평가합니다.\n- 평가자는 주관적 판단을 최소화하고, 명확한 근거에 기반하여 점수를 산출해야 합니다."}